Types of Confidence Intervals:

1. Invert two 1-sided tests
- invert one test using `alternative == "less"`, and another using `alternative == "greater"`
- for both tests, use `alpha/2`

2. Invert one 2-sided test

3. t-test confidence interval for difference in means

4. Bootstrap confidence interval

For types 1-3, we have two subtypes of confidence intervals for pooled vs. unpooled variances. Ultimately, we will have 7 types of confidence intervals to compare (since the bootstrap CI does not rely on assumptions of the population variances).

For each type of CI, simulate 7300 instances (generating new samples each time, but keeping the parameters the same). Then measure the coverage probability, i.e., the proportion of the 7300 CIs which capture the true difference in means (which is known, since the data is simulated).

In [1]:
import numpy as np
import statsmodels.stats.api as sms
import scipy.stats as stats
import math
from itertools import combinations
import threading
#from tqdm import tqdm

In [2]:
from src.search import search
from src.bootstrap import bootstrap_ci
from src.perm_test import pval

In [3]:
def get_partitions(n1, n2):
    total_length = n1 + n2

    def get_groups(idxs):
        i = 0
        idxs2 = []
        for j in idxs:
            while i < total_length and i < j:
                idxs2.append(i)
                i += 1

            if i == j:
                i += 1

        idxs2 += range(i, total_length)
        return list(idxs) + idxs2

    partitions = np.array([get_groups(idxs) for idxs in combinations(range(total_length), n1)])
    return partitions

In [4]:
def tconfint(alpha, pooled, x1, x2):
    cm = sms.CompareMeans(sms.DescrStatsW(x1), sms.DescrStatsW(x2))
    return cm.tconfint_diff(alpha, usevar="pooled" if pooled else "unequal")

In [28]:
def passToThread(n_samples):
    global n_captured, n_error
    
    for _ in range(n_samples):
        x1 = np.random.gamma(gamma1[0], gamma1[1], n1)
        x2 = np.random.gamma(gamma2[0], gamma2[1], n2)

        t99 = tconfint(0.001, pooled, x1, x2)
        t90 = tconfint(0.20, pooled, x1, x2)

        lower = search(x1, x2, partitions, t99[0], t90[0])
        if lower is None:
            n_error += 1
            continue

        upper = search(x1, x2, partitions, t90[1], t99[1])
        if upper is None:
            n_error += 1
            continue

        intervals.append((lower, upper))
        n_captured += (lower <= delta_true) * (delta_true <= upper)

In [29]:
alpha = 0.05
alternative = "less"
pooled = True

gamma1 = (2, 5)  # shape k, scale theta
gamma2 = (4, 3)
delta_true = (gamma1[0] * gamma1[1]) - (gamma2[0] * gamma2[1])  # true mean difference

n1, n2 = 12, 8
partitions = get_partitions(n1, n2)

intervals = []
n_captured = n_error = 0

In [30]:
%%time
# time how long it takes to find one confidence interval
passToThread(1)

CPU times: user 344 ms, sys: 23.7 ms, total: 368 ms
Wall time: 366 ms


In [31]:
delta_true

-2

In [32]:
n_captured, len(intervals), n_error

(1, 1, 0)

In [52]:
thread_count = 8
threads = [0] * thread_count

remaining = 1000  # 1 call per thread
batch_size = remaining // thread_count

intervals = []
n_captured = n_error = 0

In [53]:
%%time

for i in range(thread_count):
    n_samples = batch_size if i < thread_count-1 else remaining
    threads[i] = threading.Thread(target=passToThread, args=(n_samples,))
    threads[i].start()
    remaining -= n_samples
    
for thread in threads:
    thread.join()

CPU times: user 8min 38s, sys: 9.51 s, total: 8min 47s
Wall time: 1min 14s


In [54]:
n_captured, len(intervals), n_error

(978, 1000, 0)

Multithreading allows us to compute one confidence interval in \~80 ms, even faster than it took to compute the one confidence interval above (\~360 ms).

In [57]:
def total_mins(n_intervals):
    mins = n_intervals * 0.08 / 60
    hrs = mins / 60
    print("estimated time:", round(mins, 2), "mins (", round(hrs, 2), "hrs )")

In [58]:
total_mins(7300*7)

estimated time: 68.13 mins ( 1.14 hrs )


## Scrap Code

In [ ]:
# https://www.statsmodels.org/stable/generated/statsmodels.stats.weightstats.CompareMeans.html

X1, X2 = np.arange(10,21), np.arange(20,26.5,.5)

cm = sms.CompareMeans(sms.DescrStatsW(X1), sms.DescrStatsW(X2))
print(cm.tconfint_diff(0.05, usevar="pooled" if pooled else "unequal"))